# Dijkstra


In [7]:
from hotelling.generators.graph_generators import generate_grid_graph

g = generate_grid_graph(3, 3)

In [8]:
from hotelling.algorithms.branch_bound import BBTree, BaseRevenueFunction, Node, BBHeap


base_cost = 10
revenue_function = BaseRevenueFunction(base_cost)

M = 12
bb_tree = BBHeap(g, M, revenue_function)
bb_tree.run(max_iterations=50)
bb_tree.run_stat

runStatistics(open_nodes=47, reused_nodes=148, rejected_nodes=142)

In [9]:
from hotelling.algorithms.branch_bound import BBTree, BaseRevenueFunction, Node, BBHeap
from hotelling.hotelling_game.game_evaluation import evaluate_sellers

base_cost = 3
revenue_function = BaseRevenueFunction(base_cost)

M = 12
bb_tree = BBHeap(g, M, revenue_function)
bb_tree.run(max_iterations=100)
bb_tree.run_stat

runStatistics(open_nodes=47, reused_nodes=148, rejected_nodes=142)

In [10]:
bb_tree.best_value

np.float64(2.2500000000004885)

In [11]:
positions = evaluate_sellers(
    g,
    (5, 6, 7),
    M,
    "weight",
    revenue_function,extended_return=True)
positions

{'num_sellers': {6: 4, 5: 5, 7: 3},
 'lambd': np.float64(2.1000000000007955),
 'nearest_set': <VertexPropertyMap object with value type 'python::object', for Graph 0x31448ad50, at 0x314728fb0>,
 'distances': <VertexPropertyMap object with value type 'double', for Graph 0x31448ad50, at 0x314728170>,
 'revenues': defaultdict(float, {6: 9.0, 5: 10.5, 7: 7.5})}

In [6]:
from hotelling.plot_utils.widget import panel_gt_interactive
from hotelling.generators.graph_generators import generate_grid_graph, generate_line_graph

from hotelling.hotelling_game.cost_functions import BaseRevenueFunction, FeeRevenueFunction, AdditiveFeeRevenueFunction


g = generate_line_graph(5)


base_cost = 3
revenue_function = FeeRevenueFunction(base_cost, 0.5)
revenue_function = AdditiveFeeRevenueFunction(base_cost, 0.1)

panel_gt_interactive(g, revenue_function)

In [2]:
from hotelling.plot_utils.widget import panel_gt_interactive
from hotelling.generators.graph_generators import generate_grid_graph, generate_line_graph

from hotelling.hotelling_game.cost_functions import BaseRevenueFunction, FeeRevenueFunction, AdditiveFeeRevenueFunction


g = generate_line_graph(5)


base_cost = 3
revenue_function = AdditiveFeeRevenueFunction(base_cost, 1)

panel_gt_interactive(g, revenue_function)